In [1]:
import pandas as pd
import pickle
import os

In [2]:
%cd "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/tablegan" 

C:\Users\GCU\Lending_club\Data_Analysis_lending-club\tablegan


In [3]:
def cat_to_num(df, sep=',', cat_names=None):
    if cat_names is None: cat_names = []
    subs = {}
    df_num = df.copy()
    
    # TRANSFORM TO SET TO PREVENT DOUBLE FACTORIZATION
    for z in set(df_num.select_dtypes(include=['object']).columns.tolist() + cat_names):
        y, label = pd.factorize(df[z])
        subs[z] = {'y': y, 'label': label}
        df_num[z] = y
    return df_num, subs

In [4]:
def transform(ds: str, suffix: str = '', sep: str = ',', ext: str = 'csv',
              drop: list = None, cat_names: list = None, to_disk: bool = True,
              target: str = '', d_basepath: str = 'data'):
    
    if target == '':
        target = False
    if drop is None:
        drop = []
    if cat_names is None:
        cat_names = []

    base_fname = f'./{d_basepath}/{ds}/{ds}'
    source_fname = f'{base_fname}{suffix}.{ext}'
    print(f'Basepath: {base_fname}')
    print(f'Source file: {source_fname}')

    # ✅ CSV 파일 읽기 및 오류 처리
    try:
        df = pd.read_csv(source_fname, sep=sep, quotechar='"', error_bad_lines=False)
    except pd.errors.ParserError as e:
        print(f"❌ CSV 파싱 오류: {e}")
        return None, None
    except Exception as e:
        print(f"❌ CSV 파일 로드 중 오류 발생: {e}")
        return None, None

    # ✅ 문자열 내 쉼표 및 리스트 형태 처리
    def convert_to_number(value):
        if isinstance(value, str):
            value = value.strip().replace('"', '')
            try:
                # 문자열 리스트 처리: "[1.0, 2.0]" → [1.0, 2.0]
                if value.startswith('[') and value.endswith(']'):
                    return ast.literal_eval(value)
                return float(value)  # 일반 숫자 문자열 처리
            except Exception:
                return value  # 변환 실패 시 원래 값 반환
        return value

    df = df.applymap(convert_to_number)

    # ✅ 열 삭제
    df = df.drop(drop, axis=1, errors='ignore')

    # ✅ 범주형 변수 숫자 변환
    try:
        df_num, subs = cat_to_num(df, cat_names=cat_names)
    except Exception as e:
        print(f"❌ cat_to_num 함수 오류: {e}")
        return None, None

    # ✅ 변환 정보 저장
    try:
        with open(f'{d_basepath}/{ds}/subs.pkl', 'wb') as f:
            pickle.dump(subs, f)
    except Exception as e:
        print(f"❌ subs.pkl 저장 오류: {e}")

    # ✅ 타겟 분리 및 저장
    if target:
        y = df_num[target]
        df_num = df_num.drop([target], axis=1)

    if to_disk:
        if target:
            target_fname_y = f'{base_fname}_labels.csv'
            print(f'✅ 타겟 레이블 파일 저장: {target_fname_y}')
            y.to_csv(target_fname_y, sep=',', index=False)
        
        target_fname = f'{base_fname}_encoding.csv'
        print(f'✅ 변환된 데이터 저장: {target_fname}')
        df_num.to_csv(target_fname, sep=',', index=False)

    if target:
        return df_num, y, subs
    return df_num, subs


In [5]:
cat_names = ['loan_status']

In [6]:
a, b, c = transform('oversampled', suffix='', to_disk=True, target='loan_status',cat_names=cat_names)
a.head()

Basepath: ./data/oversampled/oversampled
Source file: ./data/oversampled/oversampled.csv


c:\Users\GCU\anaconda3\envs\tablegan\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  """Entry point for launching an IPython kernel.


✅ 타겟 레이블 파일 저장: ./data/oversampled/oversampled_labels.csv
✅ 변환된 데이터 저장: ./data/oversampled/oversampled_encoding.csv


,last_fico_range_high,mo_sin_old_rev_tl_op,int_rate,annual_inc,dti,installment,zip_code,revol_util,fico_range_low,total_rev_hi_lim,revol_bal,bc_util,avg_cur_bal,total_il_high_credit_limit,grade,total_pymnt,total_pymnt_inv,loan_amnt,credit_history_years,term_months
0,604.0,206.0,0.2449,67500.0,15.81,492.45,301.0,0.396,675.0,53300.0,21088.0,39.4,6661.0,79774.0,4,12284.92,12284.92,16950.0,17,60
1,559.0,315.0,0.2049,46000.0,13.75,636.04,27.0,0.471,680.0,32200.0,15158.0,44.5,1684.0,0.0,4,19664.06,19664.06,17000.0,26,36
2,499.0,161.0,0.1824,85000.0,19.41,306.30,330.0,0.447,690.0,22500.0,10057.0,66.4,6208.0,59042.0,3,3020.87,3020.87,12000.0,13,60
3,664.0,103.0,0.1825,60000.0,25.91,543.15,67.0,0.366,695.0,55200.0,20224.0,41.4,10628.0,32477.0,3,3258.90,3258.90,21275.0,9,60
4,649.0,189.0,0.1561,195000.0,10.93,482.23,335.0,0.430,685.0,48200.0,20745.0,48.8,24979.0,100840.0,2,7365.55,7365.55,20000.0,15,60


In [7]:
dataset = pd.read_csv("data/oversampled/oversampled_labels.csv")
display(len(dataset))

449334

In [9]:
for col in dataset.columns:
    print(col, "\n", dataset[col].unique(),'\n')

loan_status 
 [0] 

